In [4]:
import pandas as pd

# Load original stock data (training + testing dataset)
stock_data = pd.read_parquet("./stock_data/Processed_Stock_Data.parquet")  # Ensure this is the correct dataset

# Identify the correct test start date
test_start_date = stock_data["Date"].iloc[-len(pd.read_parquet("./models/RandomForest_No_Indicators.parquet"))]  # Last N rows used for testing

print(f"Detected Test Start Date: {test_start_date}")

Detected Test Start Date: 2021-12-30 00:00:00


In [5]:
# Load Random Forest Predictions
rf_no_ind_df = pd.read_parquet("./models/RandomForest_No_Indicators.parquet")
rf_with_ind_df = pd.read_parquet("./models/RandomForest_With_Indicators.parquet")
lstm_no_ind_df = pd.read_parquet("./models/LSTM_No_Indicators_Predictions.parquet")

# Convert numerical index to datetime
rf_no_ind_df["Date"] = pd.date_range(start=test_start_date, periods=len(rf_no_ind_df), freq="B")  # Adjust start date
rf_with_ind_df["Date"] = pd.date_range(start=test_start_date, periods=len(rf_with_ind_df), freq="B")
lstm_no_ind_df["Date"] = pd.date_range(start=test_start_date, periods=len(lstm_no_ind_df), freq="B")

# Save Updated Data
rf_no_ind_df.to_parquet("./models/RandomForest_No_Indicators.parquet", index=False)
rf_with_ind_df.to_parquet("./models/RandomForest_With_Indicators.parquet", index=False)
lstm_no_ind_df.to_parquet("./models/LSTM_No_Indicators_Predictions.parquet", index=False)

print("Fixed Date Formats for RF & LSTM")

Fixed Date Formats for RF & LSTM


In [6]:
# Load ARIMA Predictions
arima_preds_df = pd.read_parquet("./models/ARIMA_Predictions.parquet")

# Convert ARIMA Date column to datetime
arima_preds_df["Date"] = pd.to_datetime(arima_preds_df["Date"])

# Save updated ARIMA predictions
arima_preds_df.to_parquet("./models/ARIMA_Predictions.parquet", index=False)

print("Fixed Date Format for ARIMA")

Fixed Date Format for ARIMA


In [7]:
# Load all updated datasets
rf_no_ind_df = pd.read_parquet("./models/RandomForest_No_Indicators.parquet")
rf_with_ind_df = pd.read_parquet("./models/RandomForest_With_Indicators.parquet")
lstm_no_ind_df = pd.read_parquet("./models/LSTM_No_Indicators_Predictions.parquet")
arima_preds_df = pd.read_parquet("./models/ARIMA_Predictions.parquet")

# Print first few dates to check alignment
print("Random Forest No Indicators Dates:", rf_no_ind_df["Date"].head())
print("Random Forest With Indicators Dates:", rf_with_ind_df["Date"].head())
print("LSTM No Indicators Dates:", lstm_no_ind_df["Date"].head())
print("ARIMA Dates:", arima_preds_df["Date"].head())

Random Forest No Indicators Dates: 0   2021-12-30
1   2021-12-31
2   2022-01-03
3   2022-01-04
4   2022-01-05
Name: Date, dtype: datetime64[ns]
Random Forest With Indicators Dates: 0   2021-12-30
1   2021-12-31
2   2022-01-03
3   2022-01-04
4   2022-01-05
Name: Date, dtype: datetime64[ns]
LSTM No Indicators Dates: 0   2021-12-30
1   2021-12-31
2   2022-01-03
3   2022-01-04
4   2022-01-05
Name: Date, dtype: datetime64[ns]
ARIMA Dates: 0   2024-12-31
1   2025-01-01
2   2025-01-02
3   2025-01-03
4   2025-01-06
Name: Date, dtype: datetime64[ns]


In [8]:
import plotly.graph_objects as go
import numpy as np

# Load Predictions
rf_no_ind_df = pd.read_parquet("./models/RandomForest_No_Indicators.parquet")
rf_with_ind_df = pd.read_parquet("./models/RandomForest_With_Indicators.parquet")
lstm_no_ind_df = pd.read_parquet("./models/LSTM_No_Indicators_Predictions.parquet")
arima_preds_df = pd.read_parquet("./models/ARIMA_Predictions.parquet")

# Load Actual Prices
actual_prices = rf_no_ind_df["Actual"]
test_dates = rf_no_ind_df["Date"]


In [9]:
import plotly.graph_objects as go

fig = go.Figure()

# Add Actual Prices
fig.add_trace(go.Scatter(x=test_dates, y=actual_prices, mode='lines', name='Actual Price', line=dict(color='black')))

# Add Predictions
fig.add_trace(go.Scatter(x=test_dates, y=rf_no_ind_df["Predicted"], mode='lines', name='RF (No Indicators)', line=dict(dash='dot', color='blue')))
fig.add_trace(go.Scatter(x=test_dates, y=rf_with_ind_df["Predicted"], mode='lines', name='RF (With Indicators)', line=dict(dash='dot', color='red')))
fig.add_trace(go.Scatter(x=lstm_no_ind_df["Date"], y=lstm_no_ind_df["Predicted"], mode='lines', name='LSTM (No Indicators)', line=dict(dash='dash', color='green')))
fig.add_trace(go.Scatter(x=arima_preds_df["Date"], y=arima_preds_df["Forecast"], mode='lines', name='ARIMA', line=dict(dash='dashdot', color='purple')))

# Customize Layout
fig.update_layout(title="Stock Price Predictions: Actual vs Predicted",
                  xaxis_title="Date",
                  yaxis_title="Stock Price")

fig.show()